In [3]:
import pandas as pd
import numpy as np
import glob
import re
import statsmodels.api as sm
import random

class selecter():
    def __init__(self,index='對應簡稱',frequency='q',colShowSet='oriType'):
        self.convert = pd.read_excel('../data/對照清單.xlsx',engine='openpyxl')
#         self.varList =  pd.read_excel('../data/varList.xlsx',engine='openpyxl')
#         self.genr = pd.read_excel(glob.glob('../data/data_genr*.xlsx')[0])
        self.q = pd.read_excel(glob.glob('../data/data_q*.xlsx')[0],engine='openpyxl')
        self.m = pd.read_excel(glob.glob('../data/data_m*.xlsx')[0],engine='openpyxl')
        self.index = index
        self.frequency = frequency
        self.colShowSet='oriType' #smpl_all產生的dataframe中的欄位名稱是原輸入的字
        #self.smpl
        
    def show_df(self,axis,length = None):
        try:
            pd.set_option("display.max_{}".format(axis), length)
        except:
            print('axis = rows or columns')
    def show_df_reset(self,axis):
        pd.reset_option("display.max_{}".format(axis))
    def set_index_date(self,smpl):
        if smpl.index.name != 'DATE':
            smpl.set_index(['DATE'],inplace=True)
    def reset_index_date(self,df):
        if df.index.name == 'DATE':
            df.reset_index(inplace=True)
    
    
    def get_name(self,x):  # x林建甫變數-->aremos name
        if '.' in x:
            return x
        index = self.index
#         if x in self.genr.columns:
#             return x
        df_tran = self.convert
        row = df_tran.set_index(index).loc[x]
        name = row[0] # if error: have the same x
        return name
    
    def m_to_q(self,df): #之後可以用pd.to_datetime改寫
        """ 將月資料轉成季資料，取第3 6 9 12月
        input: get_x(x)"""
        freq = df.columns[1].split('.')[-1]
        def get_m(x):
            x = str(x)
            month = int(x[-2:])
            if month %3 == 0:
                return True
            return False
        df['third'] = df['DATE'].apply(lambda x:get_m(x))
        df_q = df[df['third'] == True].copy()
        df_q.pop('third')
        def rename(x):
            x = str(x)
            month = int(x[-2:])
            return '{}0{}'.format(x[:-2],month/3)[:-2]
        df_q['DATE'] = df_q['DATE'].apply(lambda x:rename(x))
        return df_q
    
    def name_to_df(self,name): # 依原始資料名稱從以下的dataframe中找出
        for df in (self.q,self.m): #,self.genr):
            if name in df.columns:
                df = df[['DATE',name]].copy()
                break
        df.dropna(inplace=True)
        df['DATE'] = df['DATE'].astype('int64')
        return df
#         if df[name].dtype != 'float64':
#             df[name] = df[name].astype('float64')
#         return df
    
    def df_ism(self,df):
        def ism(x):
            month = str(x)[-2:]
            qtype=('01','02','03','04','Q1','Q2','Q3','Q4',
                  'q1','q2','q3','q4')
            if month not in qtype:
                return True
            return False
        if df['DATE'].apply(lambda x: ism(x)).values.sum() != 0:
            x_freq= 'm'
            return True
        return False
    
    def get_x(self,x,freq=None): #  x林建甫變數-->[[time,value]]
        index = self.index
        if not freq:
            freq = self.frequency
            
        if index=='source' or '.' in x: # x已是aremos name
            name = x
#         elif x in self.genr.columns:
#             name = x
#             data = self.genr
#             df = data[['DATE',name]].copy()
#             df[name] = df[name].astype('float64')
#             return df
        else:
            name = self.get_name(x)
        df = self.name_to_df(name)
        if self.df_ism(df):
            df = self.m_to_q(df)
        if self.colShowSet =='oriType':
            df.rename(columns = {name:x},inplace=True)
            df[x] = df[x].astype('float64')
            return df
        df[name] = df[name].astype('float64')
        return df
    
    def get_dfs(self,x_list,freq=None):
        if not freq:
            freq = self.frequency
        index = self.index
        dfs =[]
        for x in x_list:
            df = self.get_x(x)
            dfs.append( df )
        return dfs
    
    def df_date_type(self,df,freq='q'):
        """q:  str: 201201 --> pd.Period: 2012Q1
           m:  str: 201201 --> pd.Period: 2012-01"""
        if type(df.index) == pd.core.indexes.period.PeriodIndex :
            return
        self.reset_index_date(df)
        def turn(x):
            if freq== 'm':
                return pd.Period(x,freq='m')
            x = str(x)
            year = int(x[:4])
            q = int(x[-2:])
            return pd.Period(freq='Q', year=year, quarter=q)
        df['DATE'] = df['DATE'].apply(lambda x:turn(x))
        self.set_index_date(df)
        return df
    def join_by_date(self,df,df2,freq='q'):
        df = self.df_date_type(df,freq)
        df2 = self.df_date_type(df2,freq)
        return pd.concat([df,df2],axis='columns')
    
    def concat_xs(self,x_list,freq=None): #這裡df裡的date轉成period格式
        """ 將get_dfs裡多個df資料依date合併"""
        index = self.index
        if type(x_list) != list:
            return self.get_x(x_list)
        dfs = self.get_dfs(x_list,freq)
        data = dfs[0]
        data= self.df_date_type(data)
        for df in dfs[1:]:
            df = self.df_date_type(df)
            data = pd.concat([data,df],axis='columns')
        pickData = data
        return pickData

    def df_non_na(self,pickData):
        pickData.dropna(inplace=True)
        start = pickData.index[0]
        length = pickData.shape[0]
        tf = pd.period_range(start, freq='Q', periods=length) == pickData.index
        if tf.sum() == length:
            return pickData
        print(pickData.columns)
        print('樣本期間內有缺漏值')
        return pickData
    
    def smpl_ori(self,x_list,freq=None):
        """ input:原始變數 """
        if len(x_list) == 0:
            return pd.DataFrame()
        index = self.index
        pickData = self.concat_xs(x_list,freq)
        smpl = self.df_non_na(pickData)
        return smpl
    
    def lag_ele(self,x):
        """ x = 'CPI(-1)' return: CPI , 1 """
        lags = int( re.search('\(-(\d+)\)',x).group(1) )
        xname = re.search('(.*)\(',x).group(1)
        return xname,lags
    def lag_df(self,x):
        xname,lags = self.lag_ele(x)
        df = self.get_x(xname)
        df.dropna(inplace=True)
        if self.colShowSet =='oriType':
            name = xname
        else:
            name = self.get_name(name)
#        ser = [np.nan]*lags + [ x for x in df[name]][:-lags]
#        df['{}(-{})'.format(name,lags)] = ser
        df['{}(-{})'.format(name,lags)] = df[name].shift(lags)
        df.pop(name)
        return df
    def add_lag_df(self,x,smpl):
        """合併一個lag期變數"""
        df = self.lag_df(x)
        df = self.df_date_type(df)
        if df.columns[0] in smpl.columns:
            print(x,'already exist')
            return smpl
        smpl = pd.concat([smpl,df],axis='columns')
        return smpl
    def smpl_with_lag(self,x_lag,smpl):
        """合併多個lag期變數"""
        for x in x_lag:
            smpl = self.add_lag_df(x,smpl)
        #smpl = self.df_non_na(smpl)
        return smpl
    
    def yoy_df(self,x):
        """ 適用季資料，ex: x='pch_CPI' """
        name = x.split('pch_')[1]
        df = self.get_x( name )
        if self.colShowSet !='oriType':
            name = self.get_name(x.split('pch_')[1])
        df['pch_'+name] = df[name].pct_change(4)
        df.pop(name)
        return df  
    def add_yoy_df(self,x,smpl):
        """適用季資料"""
        df = self.yoy_df(x)
        self.df_date_type(df)
        smpl = pd.concat([smpl,df],axis='columns')
        return smpl
    def smpl_with_yoy(self,x_yoy,smpl):
        """合併多個yoy變數"""
        for x in x_yoy:
            smpl = self.add_yoy_df(x,smpl)
        #smpl = self.df_non_na(smpl)
        return smpl

    def add_dum_period_df(self,x,smpl):
        smpl[x] = np.arange(0,len(smpl),1)
        period = re.search('\d{4}|Q\d',x).group()
        def isperiod(dum):
            if period in smpl.iloc[dum].name.strftime('%FQ%q'): # '2012Q3'
                dum = 1.0
            else:
                dum = 0.0
            return dum
        smpl[x] = smpl[x].apply(lambda dum:isperiod(dum))
        return smpl
    def smpl_with_dum_period(self,x_period,smpl):
        for x in x_period:
            self.add_dum_period_df(x,smpl)
        return smpl
    
    def log_df(self,x):
        """ x=log_GDP(-2) """
        result = re.search('[^log_]\w*\(-\d+\)',x)
#         print(result)
        if result:
        #     print(result)
            df = s.lag_df(result.group())
            name = df.columns[1]
        else:
            name = x.split('log_')[1]
#             print(name)
            df = self.get_x( name )
        if self.colShowSet !='oriType':
            name = self.get_name(x.split('log_')[1])
        df['log_'+name] = np.log(df[name])
        df.pop(name)
        return df
    def add_log_df(self,x,smpl):
        df = self.log_df(x)
        self.df_date_type(df)
        smpl = pd.concat([smpl,df],axis='columns')
        return smpl
    def smpl_with_log(self,x_log,smpl):
        for x in x_log:
            smpl = self.add_log_df(x,smpl)
        #smpl = self.df_non_na(smpl)
        return smpl
    
    def dif_df(self,x):
        difs = re.search('^d(\d+)_(.*)',x).group(1)
        oriX = re.search('^d(\d+)_(.*)',x).group(2)
        df = self.get_x( oriX )
        if self.colShowSet =='oriType':
            name = oriX
        else:
            name = self.get_name(oriX)
#         lag_name = '{}(-{})'.format(name,difs)
#         lag_df = s.lag_df(lag_name)
#         df['d{}_{}'.format(difs,name)] = df[name] - lag_df[lag_name]
        df['d{}_{}'.format(difs,name)] = df[name].diff(difs)
        df.pop(name)
        return df
    def add_dif_df(self,x,smpl):
        df = self.dif_df(x)
        self.df_date_type(df)
        smpl = pd.concat([smpl,df],axis='columns')
        return smpl
    def smpl_with_dif(self,x_dif,smpl):
        for x in x_dif:
            smpl = self.add_dif_df(x,smpl)
        #smpl = self.df_non_na(smpl)
        return smpl
    
    def first_y(self,variable_list,smpl):
        if self.colShowSet =='oriType':
            return variable_list
        first = variable_list[0]
        if '_' in first:
            vari = first.split('_')[1]
            y = '{}_{}'.format(first.split('_')[0],self.get_name(vari))
        else:
            y = self.get_name(first)
        allx = [i for i in smpl.columns]
        #print(y in allx)
        allx.remove(y)
        yx_cols = [y]+allx
        return yx_cols
    
    def smpl_all(self,variables,allExist=True):
        # used: add_dum_period_df , lag_df , df_date_type , get_x , df_non_na
        smpl = pd.DataFrame(index=pd.period_range('1920Q1','2021Q4',freq='Q-DEC'))
        for var in variables:
    #             print(var,end=',')
            x = var.split('_')[-1]
            is_lag=False
            # 季節或年 dummy
            if re.search('^D\d{4}|^Q\d',x):
                df = pd.DataFrame(index=pd.period_range('1920Q1','2021Q4',freq='Q-DEC'))
                self.add_dum_period_df(x,df)
            # 原始值 or 遞延值
            elif re.search('\(-(\d+)\)',x): #是遞延期變數
                result = re.search('\(-(\d+)\)',x)
                is_lag=True
                lag = int(result.group(1))
                df = self.get_x((x.split(result.group())[0]))
            else:
                df = self.get_x(x) #包含 data_m、data_q、genr.xlsx的變數可呼叫，date轉成 yyyy0q (整數)
            # 整數的date轉成 pd.Period並設為index
            self.df_date_type(df)
            # 有任一種 pch,d1,log 以上時
            convertFuncs = var.split('_')[:-1]
            if len(convertFuncs) > 0: 
                # ex: var='pch_d1_M2'時，上面有存出 M2的df了，再依序執行 d1、pch
                # 每 1 round 產生新一欄指定的轉換方式再刪除第一欄
                for i in range(len(convertFuncs)-1,-1,-1):
        #             print(convertFuncs[i])
                    if convertFuncs[i] == 'pch': # 年增率
                        df['pch_'+df.columns[0]] = df[df.columns[0]].pct_change(4)
                        if len(df[df['pch_'+df.columns[0]] ==np.inf]) !=0: # 存在 np.inf
                            # 取最近一期數值無限大的下一期開始
                            df = df.loc[df[df['pch_'+df.columns[0]] ==np.inf].iloc[-1].name+1:].copy() 
                    elif convertFuncs[i] == 'log': # log
                        df['log_'+df.columns[0]] = np.log(df[df.columns[0]])
                    elif re.search('^d\d+',convertFuncs[i]): # 差分
                        difs = re.search('^d(\d+)',convertFuncs[i]).group(1)
                        df['d{}_{}'.format(difs,df.columns[0])] = df[df.columns[0]].diff(difs)
                    df.pop(df.columns[0])
            if is_lag:
                df= df.shift(lag)
                df.rename(columns = {df.columns[0]:x},inplace=True)
            smpl = pd.concat([smpl,df],'columns')
        if allExist:
            self.df_non_na(smpl) # 檢查資料連續期間中有無缺漏值，並移除前後期間的NA
    #         data2[data2.T.isnull().sum()!=543] 該季有資料就留下
        return smpl

s= selecter()

In [ ]:
#     def smpl_all(self,variables,allExist=True):
#         # used: add_dum_period_df , lag_df , df_date_type , get_x , df_non_na
#         smpl = pd.DataFrame(index=pd.period_range('1920Q1','2021Q4',freq='Q-DEC'))
#         for var in variables:
# #             print(var,end=',')
#             x = var.split('_')[-1]
#             # 季節或年 dummy
#             if re.search('^D\d{4}|^Q\d',x):
#                 df = pd.DataFrame(index=pd.period_range('1920Q1','2021Q4',freq='Q-DEC'))
#                 self.add_dum_period_df(x,df)
#             # 原始值 or 遞延值
#             elif re.search('\(-(\d+)\)',x): #是遞延期變數
#                 df = self.lag_df(x)
#             else:
#                 df = self.get_x(x) #包含 data_m、data_q、genr.xlsx的變數可呼叫，date轉成 yyyy0q (整數)
#             # 整數的date轉成 pd.Period並設為index
#             self.df_date_type(df)
#             # 有任一種 pch,d1,log 以上時
#             convertFuncs = var.split('_')[:-1]
#             if len(convertFuncs) > 0: 
#                 # ex: var='pch_d1_M2'時，上面有存出 M2的df了，再依序執行 d1、pch
#                 # 每 1 round 產生新一欄指定的轉換方式再刪除第一欄
#                 for i in range(len(convertFuncs)-1,-1,-1):
#         #             print(convertFuncs[i])
#                     if convertFuncs[i] == 'pch': # 年增率
#                         df['pch_'+df.columns[0]] = df[df.columns[0]].pct_change(4)
#                         if len(df[df['pch_'+df.columns[0]] ==np.inf]) !=0: # 存在 np.inf
#                             # 取最近一期數值無限大的下一期開始
#                             df = df.loc[df[df['pch_'+df.columns[0]] ==np.inf].iloc[-1].name+1:].copy() 
#                     elif convertFuncs[i] == 'log': # log
#                         df['log_'+df.columns[0]] = np.log(df[df.columns[0]])
#                     elif re.search('^d\d+',convertFuncs[i]): # 差分
#                         difs = re.search('^d(\d+)',convertFuncs[i]).group(1)
#                         df['d{}_{}'.format(difs,df.columns[0])] = df[df.columns[0]].diff(difs)
#                     df.pop(df.columns[0])
#             smpl = pd.concat([smpl,df],'columns')
#         if allExist:
#             self.df_non_na(smpl) # 檢查資料連續期間中有無缺漏值，並移除前後期間的NA
# #         data2[data2.T.isnull().sum()!=543] 該季有資料就留下
#         return smpl

#     def smpl_all(self,variable_list,first=True,inputName=True):
#         """ input:特定變數群
#         output:其無缺漏值的最大樣本期間內的dataframe"""
#         x_list,x_lag,x_yoy,x_period,x_log,x_dif=[],[],[],[],[],[]
#         for var in variable_list:
#             if var in self.genr.columns:
#                 x_list.append(var)
# #             else:
# #                 count=0
# #                 for tran in ['pch_','d1_','log_']:
# #                     if tran in var:
# #                         count= count+1
# #                 if re.search('\(-(\d+)\)',var):
# #                     count=count+1
# #                 if count >= 2:
# #                     pass
#             elif 'log_' in var:
#                 x_log.append(var)
#             elif re.search('\(-(\d+)\)',var):
#                 x_lag.append(var)
#             elif 'pch_' in var:
#                 x_yoy.append(var)
#             elif re.search('^D\d{4}|^Q\d',var):
#                 x_period.append(var)
#             elif re.search('^d\d+_',var) :
#                 x_dif.append(var)
#             else:
#                 x_list.append(var)
# #         print(x_list,x_lag,x_yoy,x_period,x_log,x_dif)
#         smpl = self.smpl_ori(x_list)
#         smpl = self.smpl_with_log(x_log,smpl)
#         smpl = self.smpl_with_dif(x_dif,smpl)
#         smpl = self.smpl_with_lag(x_lag,smpl)
#         smpl = self.smpl_with_yoy(x_yoy,smpl)
#         self.set_index_date(smpl)
#         # self.df_date_type(smpl)
#         self.smpl_with_dum_period(x_period,smpl)
#         smpl = self.df_non_na(smpl)
# #         return smpl
#         if first:
#             yx_cols = self.first_y(variable_list,smpl)
#             smpl = smpl[yx_cols].copy()
#         return smpl

In [43]:
# s.show_df(axis='rows')

In [9]:
# variable_list = ['CPI','RX$AVG.q','JSC.m','RMIBON.q(-1)','D(LOG(GDP))','d1_CPIZF','d3_M2','JSC.m(-1)','pch_CPI.q',
#                  'pch_RX$AVG.q','Q1','D1982','Q4','log_GDP','log_STOCKTRADE'
#                 ]

# smpl = selecter().smpl_all(variable_list)
# smpl

,CPI.q,RX$AVG.q,JSC.m,D(LOG(GDP)),log_GDP.q,log_STOCKTRADE.m,d1_CPI@F.m,d3_M2@.m,RMIBON.q(-1),JSC.m(-1),pch_CPI.q,pch_RX$AVG.q,Q1,D1982,Q4
date,,,,,,,,,,,,,,,
1982Q2,58.41,38.78,19.40,0.055719,13.098550,9.406976,0.40,2181.0,13.15,20.20,0.030522,0.066557,0.0,1.0,0.0
1982Q3,59.57,39.65,19.60,0.002598,13.101148,9.641343,0.37,1835.0,12.25,19.40,0.030801,0.069887,0.0,1.0,0.0
1982Q4,59.06,40.15,21.60,0.018233,13.119381,9.271906,0.06,2436.0,9.86,19.60,0.021269,0.062169,0.0,1.0,1.0
1983Q1,59.55,39.97,21.40,-0.006339,13.113042,10.817115,0.04,2307.0,8.54,21.60,0.027610,0.054339,1.0,0.0,0.0
1983Q2,60.04,40.03,22.50,0.101277,13.214319,10.499821,-0.07,2690.0,7.30,21.40,0.027906,0.032233,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019Q1,101.84,30.83,101.60,-0.045953,15.335331,14.522358,0.07,11786.0,0.18,104.40,0.003251,0.051501,1.0,0.0,0.0
2019Q2,102.71,31.15,100.91,0.007159,15.342490,14.403483,0.51,14104.0,0.18,101.60,0.008147,0.046004,0.0,0.0,0.0
2019Q3,102.90,31.21,103.39,0.036885,15.379375,14.548420,-0.27,12798.0,0.19,100.91,0.004098,0.017275,0.0,0.0,0.0


In [2]:
# 建甫model名稱對應到的變數資料名稱
# select().varList.set_index('對應簡稱').loc['IRCUS']

檢索代號                                                 CD3M
變數中文    3-Month Certificate of Deposit: Secondary Mark...
對應中文    3-Month Certification of Deposit:Secondary Mar...
起始時間                                               1964:6
月                                                       1
備註                                                    NaN
其他代碼                                                  NaN
其他來源                                                 fred
Name: IRCUS, dtype: object

In [11]:
# smpl.index

PeriodIndex(['1982Q2', '1982Q3', '1982Q4', '1983Q1', '1983Q2', '1983Q3',
             '1983Q4', '1984Q1', '1984Q2', '1984Q3',
             ...
             '2017Q4', '2018Q1', '2018Q2', '2018Q3', '2018Q4', '2019Q1',
             '2019Q2', '2019Q3', '2019Q4', '2020Q1'],
            dtype='period[Q-DEC]', name='date', length=152, freq='Q-DEC')

# --------------------------------------------------- 